# Import

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
from glob import glob
import matplotlib.pyplot as plt
from scipy.special import factorial
from scipy.optimize import minimize
import uproot as ur
import ROOT as rt
import root_numpy as rtnp
import ROOT.RooFit as rf
from scipy.special import erf
import sys, os, yaml
sys.path.append('../lib')
sys.path.append('../analysis')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, SetMaxToMaxHist, make_ratio_plot
from lumi_utilities import getLumiByTrigger
from cebefo_style import Set_2D_colz_graphics
from progressBar import ProgressBar
from array import array
from categoriesDef import categories

from analysis_utilities import drawOnCMSCanvas, getEff, DSetLoader
from pileup_utilities import pileupReweighter
from pT_calibration_reader import pTCalReader
from lumi_utilities import getLumiReport

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Internal"

donotdelete = []

Welcome to JupyROOT 6.12/07


In [2]:
from prettytable import PrettyTable

In [3]:
rt.gErrorIgnoreLevel = rt.kError
rt.RooMsgService.instance().setGlobalKillBelow(rt.RooFit.ERROR)

In [4]:
m_D0 = 1.86483 #1e-3*Particle.from_string('D0').mass
print m_D0
m_Dst = 2.01026 #1e-3*Particle.from_string('D*+').mass
print m_Dst
m_B0 = 5.27963 #1e-3*Particle.from_string('B0').mass
print m_B0
m_pi = 0.13957018 #1e-3*Particle.from_string('pi+').mass
print 'mD* - mD0 =', m_Dst - m_D0

1.86483
2.01026
5.27963
mD* - mD0 = 0.14543


# Get MC

In [6]:
cat = categories['high']

In [8]:
#They all have to be produced with the same pileup
MCsample = {
'mu' : DSetLoader('Bd_MuNuDst', candDir='ntuples_B2DstMu_wOC'),
    
# 'old_mu' : DSetLoader('B0_MuNuDmst_PUc0'),
# 'tau' : DSetLoader('B0_TauNuDmst_PUc0'),
# 'Hc' : DSetLoader('B0_DmstHc_PUc0'),
# 'DstPip' : DSetLoader('Bp_MuNuDstst_Pip_PUc0'),
# 'DstPi0' : DSetLoader('B0_MuNuDstst_Pi0_PUc0'),
# 'DstPi0' : DSetLoader('B0_DmstPi0MuNu_PUc0'),
# 'DstPipPi0' : DSetLoader('Bp_MuNuDstst_PipPi0_PUc0'),
# 'DstPipPim' : DSetLoader('B0_MuNuDstst_PipPim_PUc0'),
# 'DstPi0Pi0' : DSetLoader('B0_MuNuDstst_Pi0Pi0_PUc0'),
}
df = {}
for n, s in MCsample.iteritems():
    df[n] = pd.DataFrame(rtnp.root2array(s.skimmed_dir + '/{}_trkCtrl_bare.root'.format(cat.name)))

In [ ]:
for dn in ['mu', 'DstPip']:
    if not dn in MCsample.keys(): continue
    print dn
    d = df[dn]
    sel = d['N_goodAddTks'] == 1
    e, de = getEff(np.sum(d['MC_tkFlag_0'][sel] > 0), np.sum(sel))
    print 'Tot: {:.1f} +/- {:.1f} %'.format( 100*e, 100*de )
    
    selp = np.logical_and(sel, d['tkCharge_0'] > 0)
    e, de = getEff(np.sum(d['MC_tkFlag_0'][selp] > 0), np.sum(selp))
    print 'Plus: {:.1f} +/- {:.1f} %'.format(100*e, 100*de)
    selm = np.logical_and(sel, d['tkCharge_0'] < 0)
    e, de = getEff(np.sum(d['MC_tkFlag_0'][selm] > 0), np.sum(selm))
    print 'Minus: {:.1f} +/- {:.1f} %'.format(100*e, 100*de)
    
    
    print '\n'+30*'-'+ '\n'

### Some MC plots

In [ ]:
d = df['DstPip']

In [ ]:
sel = np.logical_and(d['N_goodAddTks'] == 1, d['tkCharge_0'] > 0)
h = create_TH1D(d['tkMassHad_0'][sel], binning=[50, 2, 3.5], axis_title=['Hadronic Mass [GeV]', 'normalized entries'], scale_histo='norm')
c = drawOnCMSCanvas(CMS_lumi, [h])

In [ ]:
np.sum(np.logical_and(d['MC_tkFlag_0'] == 1, sel))/float(np.sum(sel))

In [ ]:
selDstPi = np.logical_and(df['DstPip']['N_goodAddTks'] == 1, df['DstPip']['tkCharge_0'] > 0)
selDstPi = np.logical_and(df['DstPip']['MC_tkFlag_0'] == 1, selDstPi)
selMu = np.logical_and(df['mu']['N_goodAddTks'] == 1, df['mu']['tkCharge_0'] > 0)
selMu = np.logical_and(df['mu']['MC_tkFlag_0'] == 1, selMu)

In [ ]:
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], df['DstPip']['MC_tk_dpt_0'][selDstPi])), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.03), 30)], 
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} /p_{T}', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')
c.SetLogy()

h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], df['mu']['MC_tk_dpt_0'][selMu])), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.03), 30)], 
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} /p_{T}', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')
c1.SetLogy()

In [ ]:
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], df['DstPip']['MC_tk_dphi_0'][selDstPi])), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002), 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta #phi', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')
c.SetLogy()

h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], df['mu']['MC_tk_dphi_0'][selMu])), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002), 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta #phi', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')
c1.SetLogy()

In [ ]:
print np.sum(np.logical_and(df['DstPip']['MC_tk_dphi_0'] < 0.001, selDstPi))/float(np.sum(selDstPi))
print np.sum(np.logical_and(df['mu']['MC_tk_dphi_0'] < 0.001, selMu))/float(np.sum(selMu))

In [ ]:
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], df['DstPip']['MC_tk_deta_0'][selDstPi])), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002), 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta #eta', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')
c.SetLogy()

h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], df['mu']['MC_tk_deta_0'][selMu])), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002), 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta #eta', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')
c1.SetLogy()

In [ ]:
dRDstPi = np.hypot(df['DstPip']['MC_tk_deta_0'][selDstPi], df['DstPip']['MC_tk_dphi_0'][selDstPi])
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], dRDstPi)), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002*np.sqrt(2)), 30)],
                axis_title=['Hadronic Mass [GeV]', '#DeltaR', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')
c.SetLogy()

dRMu = np.hypot(df['mu']['MC_tk_deta_0'][selMu], df['mu']['MC_tk_dphi_0'][selMu])
h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], dRMu)), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.logspace(-5, np.log10(0.002*np.sqrt(2)), 30)],
                axis_title=['Hadronic Mass [GeV]', '#DeltaR', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')
c1.SetLogy()

In [ ]:
print np.sum(dRDstPi < 0.002)/float(np.sum(selDstPi))
print np.sum(dRMu < 0.002)/float(np.sum(selMu))

In [ ]:
dRDstPi = (df['DstPip']['tkPt_0'][selDstPi] - df['DstPip']['mu_pt'][selDstPi])/df['DstPip']['mu_pt'][selDstPi]
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], dRDstPi)), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.linspace(-1, 1, 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #mu', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')

dRMu = (df['mu']['tkPt_0'][selMu] - df['mu']['mu_pt'][selMu])/df['mu']['mu_pt'][selMu]
h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], dRMu)), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.linspace(-1, 1, 50)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #mu', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')

In [ ]:
dRDstPi = (df['DstPip']['tkPt_0'][selDstPi] - df['DstPip']['pi_pt'][selDstPi])/df['DstPip']['pi_pt'][selDstPi]
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], dRDstPi)), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.linspace(-1, 1, 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #pi', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')

dRMu = (df['mu']['tkPt_0'][selMu] - df['mu']['pi_pt'][selMu])/df['mu']['pi_pt'][selMu]
h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], dRMu)), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.linspace(-1, 1, 50)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #pi', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')

In [ ]:
dRDstPi = (df['DstPip']['tkPt_0'][selDstPi] - df['DstPip']['pis_pt'][selDstPi])/df['DstPip']['pis_pt'][selDstPi]
h = create_TH2D(np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], dRDstPi)), 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.linspace(-5, 5, 30)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #pi', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')

dRMu = (df['mu']['tkPt_0'][selMu] - df['mu']['pis_pt'][selMu])/df['mu']['pis_pt'][selMu]
h1 = create_TH2D(np.column_stack((df['mu']['tkMassHad_0'][selMu], dRMu)), 'mu',
                binning=[np.linspace(2, 3.5, 50), np.linspace(-5, 5, 50)],
                axis_title=['Hadronic Mass [GeV]', '#Delta p_{T} w.t.r. #pi soft', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')

In [ ]:
aux = np.column_stack((df['DstPip']['tkMassHad_0'][selDstPi], df['DstPip']['tkPt_0'][selDstPi]))
h = create_TH2D(aux, 'DstPip', 
                binning=[np.linspace(2, 3.5, 50), np.logspace(np.log10(0.2), np.log10(5.0), 40)],
                axis_title=['Hadronic Mass [GeV]', 'Track p_{T} [GeV]', '']
               )
c = drawOnCMSCanvas(CMS_lumi, [h], ['colz'], tag='DstPip')

aux = np.column_stack((df['mu']['tkMassHad_0'][selMu], df['mu']['tkPt_0'][selMu]))
h1 = create_TH2D(aux, 'mu',
                binning=h.binning,
                axis_title=['Hadronic Mass [GeV]', 'Track p_{T} [GeV]', '']
               )
c1 = drawOnCMSCanvas(CMS_lumi, [h1], ['colz'], tag='mu')

## PdgId Plots

In [ ]:
pdgid2LatexName = {
0: 'N/A',
1: 'd',
-1: '\\bar{d}',
2: 'u',
-2: '\\bar{u}',
3: 's',
-3: '\\bar{s}',
4: 'c',
-4: '\\bar{c}',
5: 'b',
-5: '\\bar{b}',
11: 'e^{-}',
-11: 'e^{-}',
13: '\mu^{-}',
-13: '\mu^{+}',
15: '\\tau^{-}',
-15: '\\tau^{+}',
21: 'g',
22: '\gamma',  
111: '\pi_{0}',
113: '\\rho_{0}',
-113: '\\bar{\\rho}_{0}',
211: '\pi^{+}',
-211: '\pi^{-}',
213: '\\rho^{+}',
-213: '\\rho^{-}',
221: '\eta',
-221: '\\bar{eta}',
223: '\omega',
-223: '\\bar{omega}',
313: 'K^{*}_{0}',
-313: '\\bar{K}^{*}_{0}',
315: 'K_{2}^{*0}',
-315: '\\bar{K}_{2}^{*0}',
321: 'K^{+}',
-321: 'K^{-}',
323: 'K^{*+}',
-323: 'K^{*-}',
331: '\eta^{\prime}',
-331: '\\bar{eta}^{\prime}',
333: '\phi',
-333: '\\bar{phi}',
411: 'D^{+}',
-411: 'D^{-}',
413: 'D^{*+}',
-413: 'D^{*-}',
415: 'D_{2}^{*+}',
-415: 'D_{2}^{*-}',
421: 'D_{0}',
-421: '\\bar{D}_{0}',
423: 'D_{0}^{*}',
-423: '\\bar{D}_{0}^{*}',
425: 'D_{2}^{*0}',
-425: '\\bar{D}_{2}^{*0}',
431: 'D_{s}^{+}',
-431: 'D_{s}^{-}',
511: 'B_{0}',
-511: '\\bar{B}_{0}',
513: 'B_{0}^{*}',
-513: '\\bar{B}_{0}^{*}',
521: 'B^{+}',
-521: 'B^{-}',
523: 'B^{*+}',
-523: 'B^{*-}',
541: 'B_{c}^{+}',
-541: 'B_{c}^{-}',
543: 'B_{c}^{*+}',
-543: 'B_{c}^{*-}',
2212: 'p',
-2212: '\\bar{p}',
3112: '\Sigma^{-}',
-3112: '\Sigma^{+}',
3312: '\Xi^{-}',
-3312: '\Xi^{+}',
4122: '\Lambda_{c}^{+}',
-4122: '\Lambda_{c}^{-}',
4222: '\Lambda_{c}^{++}',
-4222: '\Lambda_{c}^{--}',
4224: '\Lambda_{c}^{*++}',
-4224: '\Lambda_{c}^{*--}',
10313: 'K_{1}^{0}',
-10313: '\\bar{K}_{1}^{0}',
10423: 'D_{1}^{0}',
-10423: '\\bar{D}_{1}^{0}',
20213: 'a_{1}^{+}',
-20213: 'a_{1}^{-}',
20423: 'D_{1}^{*}',
-20423: '\\bar{D}_{1}^{*0}'
}
# pdgid2LatexName

In [ ]:
def plotPdgId(dname, varPdgId, sel=None, logy=False, size=(8,6), 
              threshold=1e-4, ylim=None, ylabel='Events fraction',
              verbose=False
             ):
    if sel is None:
        sel = np.ones_like(df[dname][varPdgId]).astype(np.bool)

    pdgId = {}
    auxList = list(df[dname][varPdgId][sel])
    nTot = 0
    for n in set(auxList):
        pdgId[n] = auxList.count(n)
        nTot += pdgId[n]
    print pdgId
    
    for k in pdgId.keys():
        pdgId[k] = pdgId[k]/float(nTot)
        if pdgId[k] < threshold:
            pdgId.pop(k)
            if verbose: print k, 'removed'

    weights = []
    sortedPdgId = np.sort(pdgId.keys())
    for k in sortedPdgId:
        weights.append(pdgId[k])


    n = len(weights)
    fig = plt.figure(figsize=size)
    plt.hist(np.arange(0, n), weights=np.array(weights), range=(-0.5, n-0.5), bins=n);
    plt.xticks(np.arange(0, n), [r'$'+pdgid2LatexName[x]+'$' for x in sortedPdgId])
    plt.ylabel(ylabel)
    if ylim is None:
        plt.ylim(threshold, 1)
    else:
        plt.ylim(ylim[0], ylim[1])
    if logy: plt.yscale('log')
    return fig

In [ ]:
plt.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.sans-serif': ['Verdana', 'Arial']})

### Study plots

In [ ]:
sel = np.logical_and(df['DstPip']['N_goodAddTks'] == 1, df['DstPip']['MC_tkFlag_0'] == 1)
plotPdgId('DstPip', 'MC_tkPdgId_0', sel=sel, logy=True);

In [ ]:
sel = np.logical_and(df['DstPip']['N_goodAddTks'] == 1, df['DstPip']['MC_tkFlag_0'] == 1)
fig = plotPdgId('DstPip', 'MC_tkMotherPdgId_0', sel=sel, size=(12,4), logy=True)

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 1, df['mu']['MC_tkFlag_0'] == 1)
plotPdgId('mu', 'MC_tkPdgId_0', sel=sel);

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 1, df['mu']['MC_tkFlag_0'] == 1)
fig = plotPdgId('mu', 'MC_tkMotherPdgId_0', sel=sel, size=(12,4), threshold=1e-3, logy=True)

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 1, df['mu']['MC_tkFlag_0'] == 1)
fig = plotPdgId('mu', 'MC_tkMotherMotherPdgId_0', sel=sel, size=(12,4))

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 2, df['mu']['MC_tkFlag_0'] == 1)
fig = plotPdgId('mu', 'MC_tkMotherPdgId_0', sel=sel, size=(12,4))

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 2, df['mu']['MC_tkFlag_1'] == 1)
fig = plotPdgId('mu', 'MC_tkMotherPdgId_1', sel=sel, size=(12,4))

In [ ]:
sel = np.logical_and(df['mu']['N_goodAddTks'] == 2, df['mu']['MC_tkFlag_0'] == 1)
fig = plotPdgId('mu', 'MC_tkMotherMotherPdgId_0', sel=sel, size=(12,4))

In [ ]:
sel = np.logical_and(df['DstPip']['N_goodAddTks'] == 2, df['DstPip']['tkCharge_0']+df['DstPip']['tkCharge_1'] == 0)
sel = np.logical_and(sel, df['DstPip']['tkCharge_1'] < 0)
sel = np.logical_and(sel, df['DstPip']['MC_tkFlag_1'] == 1)
fig = plotPdgId('DstPip', 'MC_tkMotherPdgId_1', sel=sel, size=(12,4))

In [ ]:
# sel = np.logical_and(df['Hc']['N_goodAddTks'] == 2, df['Hc']['tkCharge_0']+df['Hc']['tkCharge_1'] == 0)
# sel = np.logical_and(sel, df['Hc']['MC_tkFlag_1'] == 1)
# fig = plotPdgId('Hc', 'MC_tkMotherPdgId_1', sel=sel, size=(12,4))

### AN plots

In [ ]:
for dn in ['mu', 'DstPip']:
    d = df[dn]
    sel = np.logical_and(d['N_goodAddTks'] == 1, d['MC_tkFlag_0'] == 1)
    fig = plotPdgId(dn, 'MC_tkMotherPdgId_0', sel=np.logical_and(sel, d['tkCharge_0'] > 0),
                    size=(12,4), threshold=1e-3, logy=True)
    xmin, xmax = plt.xlim()
    plt.text(xmin + 0.05*(xmax - xmin) , 0.5, 'Q = +1')
    
    fig = plotPdgId(dn, 'MC_tkMotherPdgId_0', sel=np.logical_and(sel, d['tkCharge_0'] < 0),
                    size=(12,4), threshold=1e-3, logy=True)
    xmin, xmax = plt.xlim()
    plt.text(xmin + 0.05*(xmax - xmin) , 0.5, 'Q = -1')

## All data

In [ ]:
date = '201101'
RDloc = '../data/cmsRD/skimmed/B2DstMu_B0_{}_{}_trkCtrl_corr.root'.format(date, cat.name)
df['data'] = pd.DataFrame(rtnp.root2array(RDloc))
datasets_loc = glob('../data/cmsRD/ParkingBPH*/*2018*B2DstMu_{}*'.format(date))
lumi_tot = getLumiByTrigger(datasets_loc, cat.trg, verbose=True)
CMS_lumi.integrated_lumi = lumi_tot

## MC comparison

In [ ]:
puRew = pileupReweighter(MCsample['mu'].skimmed_dir + '/{}_trkCtrl_bare.root'.format(cat.name), 
                         'hAllNTrueIntMC', 
                         trg=cat.trg)
df['mu']['wPU'] = puRew.getPileupWeights(df['mu']['MC_nInteractions'])

In [ ]:
cal_pT = pTCalReader(
    calibration_file='../data/calibration/B0pTspectrum/polyCoeffWeights_{}_v20.pkl'.format(cat.name)
)
df['mu']['wBpt'] = cal_pT.getWeights(df['mu']['MC_B_pt'], 0)

In [ ]:
df['mu']['w'] = df['mu']['wBpt']*df['mu']['wPU']

In [ ]:
baseSelMC = np.logical_and(df['mu']['N_goodAddTks'] == 1, df['mu']['tkCharge_0'] == -1)
baseSelRD = np.logical_and(df['data']['N_goodAddTks'] == 1, df['data']['tkCharge_0'] == -1)

In [ ]:
def plotComp(var, binning, xtitle, logy=False):
    leg = rt.TLegend(0.6, 0.6, 0.95, 0.9)
    leg.SetBorderSize(0)
    leg.SetTextFont(42)

    hlist = []

    h = create_TH1D(df['data'][var][baseSelRD], binning=binning, 
                    opt='underflow+overflow',
                    axis_title=[xtitle, 'Normalized entries'], scale_histo='norm')
    leg.AddEntry(h, 'data', 'lep')
    hlist.append(h)

    sel = np.logical_and(baseSelMC, df['mu']['MC_tkFlag_0'] == -1)
    h = create_TH1D(df['mu'][var][sel], binning=h.binning,
                    weights=df['mu']['w'][sel],
                    opt='underflow+overflow',
                    scale_histo='norm', color=0)
    leg.AddEntry(h, 'B #rightarrow D*#mu#nu (PU only)', 'lep')
    hlist.append(h)

    sel = np.logical_and(baseSelMC, df['mu']['MC_tkFlag_0'] == 1)
    h = create_TH1D(df['mu'][var][sel], binning=h.binning,
                    weights=puWeights[sel],
                    opt='underflow+overflow',
                    scale_histo='norm', color=1)
    leg.AddEntry(h, 'B #rightarrow D*#mu#nu (PV only)', 'lep')
    hlist.append(h)

    h = create_TH1D(df['mu'][var][baseSelMC],
                    weights=puWeights[baseSelMC],
                    opt='underflow+overflow',
                    binning=h.binning, scale_histo='norm', color=2)
    leg.AddEntry(h, 'B #rightarrow D*#mu#nu', 'lep')
    hlist.append(h)

    m = SetMaxToMaxHist(hlist)
    c = drawOnCMSCanvas(CMS_lumi, hlist, 'same')
    if logy:
        hlist[0].GetYaxis().SetRangeUser(logy, m)
        c.SetLogy()
    leg.Draw()
    c.dnd = [leg, hlist]
    return c

In [ ]:
plotComp('tkMassHad_0', [61, 2.1, 3.2], 'Hadronic Mass [GeV]')

In [ ]:
c = plotComp('tkPt_0', [61, 0.5, 5.0], 'Additional track p_{T} [GeV]', logy=1e-3)

In [ ]:
c = plotComp('tkEta_0', [61, -2.5, 2.5], 'Additional track #eta')

In [ ]:
var = 'tkPt_0'

binning = np.logspace(np.log10(0.5), np.log10(5.0), 40)
binning = [61, 0.3, 5.0]
binning = array('d', list(np.arange(0.5, 3.02, 0.07)) + list(np.logspace(np.log10(3.02), np.log10(5.0), 10)))
hRD = create_TH1D(df['data'][var][baseSelRD], binning=binning)
hRD.Sumw2()

puWeights = puRew.getPileupWeights(df['mu']['N_vtx'])

sel = np.logical_and(baseSelMC, df['mu']['MC_tkFlag_0'] == -1)
hMC_PU = create_TH1D(df['mu'][var][sel], binning=hRD.binning, color=0, weights=puWeights[sel])


sel = np.logical_and(baseSelMC, df['mu']['MC_tkFlag_0'] == 1)
hMC_PV = create_TH1D(df['mu'][var][sel], binning=hRD.binning, color=1, weights=puWeights[sel],
                     axis_title=['Additional track p_{T} [GeV]', 'Events']
                    )

r_pre = hMC_PV.Integral()/(hMC_PV.Integral() + hMC_PU.Integral())
print 'fPV (pre-fit): {:.3f}'.format(r_pre)
hMC_PU.Scale(1./hMC_PU.Integral())
hMC_PV.Scale(1./hMC_PV.Integral())

pt = rt.RooRealVar('pt', 'p_{T}', 0.5, 5.0, 'GeV')

data = rt.RooDataHist('Data', 'Data', rt.RooArgList(pt), hRD)
N_data = hRD.Integral()

dh_PU = rt.RooDataHist('dh_PU', 'hPU', rt.RooArgList(pt), hMC_PU)
pdf_PU = rt.RooHistPdf('pdf_PU', 'pdf_PU', rt.RooArgSet(pt), dh_PU)
n_PU = rt.RooRealVar('n_PU', 'N_{PU}', N_data*0.5, 0, N_data)

dh_PV = rt.RooDataHist('dh_PV', 'hPV', rt.RooArgList(pt), hMC_PV)
pdf_PV = rt.RooHistPdf('pdf_PV', 'pdf_PV', rt.RooArgSet(pt), dh_PV)
n_PV = rt.RooRealVar('n_PV', 'N_{PV}', N_data*0.5, 0, N_data)

pdf_tot = rt.RooAddPdf('pdftot', 'pdftot', rt.RooArgList(pdf_PU, pdf_PV), rt.RooArgList(n_PU, n_PV))
fr = pdf_tot.fitTo(data, rf.PrintLevel(-1), rf.Save())

dof = hRD.GetNbinsX()-fr.floatParsFinal().getSize()
chi2 = rt.RooChi2Var("chi2", 'chi2', pdf_tot, data)
pval = rt.ROOT.Math.chisquared_cdf_c(chi2.getVal(), dof)
print 'chi2: {:.1f}/{:.0f} {:.3f}'.format(chi2.getVal(), dof, pval)

r = n_PV.getVal()/(n_PV.getVal() + n_PU.getVal())

dr = np.hypot(n_PV.getVal()*n_PU.getError(), n_PU.getVal()*n_PV.getError())/((n_PV.getVal() + n_PU.getVal())**2)
print 'fPV = {:.3f} +/- {:.3f} [{:.1f}%]'.format(r, dr, 100*dr/r)
print 'PV weight: {:.1f} +/- {:.1f}'.format(r/r_pre, dr/r_pre)

hMC_PU.Scale(n_PU.getVal())
hMC_PV.Scale(n_PV.getVal())
hMC_PV.Add(hMC_PU)
hMC_PU.Sumw2(0)
hMC_PU.SetFillStyle(1)
hMC_PV.Sumw2(0)
hMC_PV.SetFillStyle(1)
c = drawOnCMSCanvas(CMS_lumi, [hMC_PV, hMC_PU, hRD], 'same')

leg = rt.TLegend(0.6, 0.6, 0.95, 0.9)
leg.SetBorderSize(0)
leg.SetTextFont(42)
leg.AddEntry(hRD, 'data', 'lep')
leg.AddEntry(hMC_PU, 'B #rightarrow D*#mu#nu (PU only)', 'f')
leg.AddEntry(hMC_PV, 'B #rightarrow D*#mu#nu (PV only)', 'f')
leg.Draw()
c.SetLogy()